In [4]:
import xgboost as xgb
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
import pyspark.sql.functions as SF

In [9]:
df_test = spark.read.format("csv").option("header", "true").load("/home/maria/Documents/ML/GradientBoost/flight_delays_test.csv")
df1 = spark.read.format("csv").option("header", "true").load("/home/maria/Documents/ML/GradientBoost/flight_delays_train.csv")

In [10]:
#UDF para tratamento nos Nomes e conseguir retornar apenas os sobrenomes
@SF.udf("Int")
def fix_string(aux):
    
    if aux is None:
        return None
    else: 
        aux1= aux.split("-")
        return int(aux1[1])

In [11]:
df_test1=df_test.withColumn("dep_delayed_15min",SF.lit("0"))
df_test=df_test.withColumn("Month",fix_string("Month"))
df_test=df_test.withColumn("DayofMonth",fix_string("DayofMonth"))
df_test=df_test.withColumn("DayOfWeek",fix_string("DayOfWeek"))
df_test=df_test.withColumn("Distance", df_test["Distance"].cast("Int"))
df_test=df_test.withColumn("DepTime", df_test["DepTime"].cast("Int"))
df_test1=df_test.withColumn("dep_delayed_15min",SF.lit("0"))




In [12]:
df1=df1.withColumn("Month",fix_string("Month"))
df1=df1.withColumn("DayofMonth",fix_string("DayofMonth"))
df1=df1.withColumn("DayOfWeek",fix_string("DayOfWeek"))
df1=df1.withColumn("dep_delayed_15min", SF.when(SF.col("dep_delayed_15min") == "N",0)
      .when(SF.col("dep_delayed_15min") == "Y",1)
      .otherwise("Unknown"))
df1=df1.withColumn("Distance", df1["Distance"].cast("Int"))
df1=df1.withColumn("DepTime", df1["DepTime"].cast("Int"))


In [13]:
df_union=df1.union(df_test1)
aux=df_union.select("Dest")
aux2=df_union.select("Origin")
df_union1=df_union.select('UniqueCarrier').distinct()
df_union2=aux.union(aux2).distinct()

In [14]:
df_test1=df_test.toPandas()
df11=df1.toPandas()
df_union3=df_union1.toPandas()
df_union4=df_union2.toPandas()

In [15]:
df_train1=df11.drop("dep_delayed_15min",axis=1)
df_train_target1=df11["dep_delayed_15min"]

In [16]:
le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()

le1.fit_transform(df_union3["UniqueCarrier"])
le2.fit_transform(df_union4["Dest"])

df_train1["UniqueCarrier"]=le1.transform(df_train1["UniqueCarrier"])
df_train1["Origin"]=le2.transform(df_train1["Origin"])
df_train1["Dest"]=le2.transform(df_train1["Dest"])

df_test1["UniqueCarrier"]=le1.transform(df_test1["UniqueCarrier"])
df_test1["Origin"]=le2.transform(df_test1["Origin"])
df_test1["Dest"]=le2.transform(df_test1["Dest"])


In [149]:
df_train1.describe()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
count,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000
mean,6.537640,15.703850,3.95183,1341.523880,13.045040,149.365770,149.293960,729.39716
std,3.413447,8.793931,1.99164,476.378445,6.566272,80.935548,81.130353,574.61686
min,1.000000,1.000000,1.00000,1.000000,1.000000,0.000000,0.000000,30.00000
25%,4.000000,8.000000,2.00000,931.000000,7.000000,81.000000,81.000000,317.00000
50%,7.000000,16.000000,4.00000,1330.000000,14.000000,159.000000,159.000000,575.00000
75%,9.000000,23.000000,6.00000,1733.000000,19.000000,214.000000,214.000000,957.00000
max,12.000000,31.000000,7.00000,2534.000000,22.000000,306.000000,306.000000,4962.00000


In [17]:
model = xgb.XGBClassifier(learning_rate=0.05,max_depth= 10,min_child_weight= 1,n_estimators= 200,colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,)

model.fit(df_train1,df_train_target1)
Y_pred=model.predict_proba(df_train1)[:, 1]
roc_auc_score(df_train_target1, Y_pred)


0.919131696342652

In [159]:
teste_pred=model.predict_proba(df_test1)[:, 1]

pd.Series(teste_pred, name='dep_delayed_15min').to_csv('teste_LightGBM.csv', index_label='id', header=True)